## Initiate

#### Configurações gerais

In [1]:
from Petrobras_AI_Agents import llmClient_AzureOpenAI as llmClient
from Petrobras_AI_Agents import ChatDatabaseManager_Postgres as ChatMemory
from Petrobras_AI_Agents import KnowledgeBaseManager_Databricks as Databricks_KnowledgeBase
from Petrobras_AI_Agents import KnowledgeBaseManager_SQLite as PersonalCollection
from Petrobras_AI_Agents import DatabaseManager_SQLite, DatabaseManager_Databricks
from Petrobras_AI_Agents import AgentWork, BaseAgent, ChatInterpretorAgent, KnowledgeBaseAgent, MemoryAgent, DatabaseExpertAgent
from Petrobras_AI_Agents import databricks_connector

from Petrobras_AI_Agents import read_file
from Petrobras_AI_Agents.TOOLS import basic_calculator, run_python_code, search_web

import os
from decouple import config

In [2]:
username = os.getlogin().upper()
username = None
language = "pt-br"
local_path = lambda path_tipe, file: os.path.normpath(os.path.join("C:/", "Users", username.lower(), "OneDrive - PETROBRAS", "Documents", "chat_contabilidade", path_tipe, file))

In [3]:
# Nome dos agentes a serem criados
chat_interpretor_agent_name     = "Chat Analista"
databricks_knowledge_agent_name = "Documentos Databricks"
databricks_analytics_agent_name = "Analytics Databricks"
personal_knowledge_agent_name   = "Documentos Pessoais"
personal_analytics_agent_name   = "Analytics Pessoal"

In [4]:
agent_work = AgentWork(
    start_agent = chat_interpretor_agent_name,
    language = language,
    llm = llmClient(
        model_text  ="gpt-4o-petrobras",
        model_emb   ="text-embedding-3-large-petrobras",
        api_version ="2024-06-01", #"2024-08-01-preview",
        base_url    ="https://apit.petrobras.com.br/ia/openai/v1/openai-azure/openai",
        cert_file   ="servicoca-petrobras-com-br-chain.pem",
        api_key     =config("AZURE_OPENAI_API_KEY"),
        temperature=0.1),
    chat_memory = ChatMemory( 
        db_path     = config("POSTGRES_PASSWORD"),
        user        = username,
        language    = language),
)

### Agentes Padrão

#### ChatInterpretorAgent

In [7]:
chat_interpretor_agent = ChatInterpretorAgent(
    agent_name = chat_interpretor_agent_name,
    work_instance = agent_work,
    human_in_the_loop = False,
    # next_agent_list = ['colocar o nome dos agentes']  por padrão esse agente pode se comunicar com todos os outros agentes
    )

### Agentes Databricks

#### Databricks_KnowledgeBase

In [6]:
config_json = os.path.join("config_json", "databricks", "config_databricks_doc_collection.json")

In [7]:
# Apenas necessário se ainda não existir config_file (se ja existir, esse comando será automaticamente ignorado)

Databricks_KnowledgeBase.create_config_file(
    config_json                 = config_json,
    connection_server_hostname  = "adb-671799829240675.15.azuredatabricks.net",
    connection_http_path        = "/sql/1.0/warehouses/1fd972f888afd086",
    users_access_table_name     = "dtcore_prd.aida.acesso_usuario",
    users_access_profile_prefix = "GDB_DLKC_PRD_PFL_",
    db_functions_euclidean      = "dt0047_prd.db_functions.euclidean_distance",
    db_functions_cosine         = "dt0047_prd.db_functions.cosine_distance",
    source_col_name             = "source",
    file_col_name               = "file_as_str",
    context_col_name            = "page_content",
    rag_col_name                = "vector_embedding_3large",
    metadata_col_name           = "metadata_dictionary",
    file_base_id_col_name       = "id",
    vec_base_file_id_col_name   = "file_id"
    )

In [8]:
# Adiciona Coleções de Dados, quantas desejar (se já estiver cadastrada no config_json não precisa rodar)

Databricks_KnowledgeBase.add_data_source(
    process_name    = "intimacoes_sefaz_rj",
    file_base       = "dt0047_prd.chat_bot.intimacoes_fiscais_pgov_sefaz_rj_file_base",
    vec_base        = "dt0047_prd.chat_bot.intimacoes_fiscais_pgov_sefaz_rj_vec_base"
)
Databricks_KnowledgeBase.add_data_source(
    process_name    = "normas_contabeis",
    file_base       = "dt0047_prd.chat_bot.normas_contabeis_guia_de_normas_file_base",
    vec_base        = "dt0047_prd.chat_bot.normas_contabeis_guia_de_normas_vec_base"
)
Databricks_KnowledgeBase.add_data_source(
    process_name    = "apontamento_hh_parcerias",
    file_base       = "dt0047_prd.chat_bot.parcerias_apontamentohoras_file_base",
    vec_base        = "dt0047_prd.chat_bot.parcerias_apontamentohoras_vec_base"
)

In [9]:
doc_manager_databricks = Databricks_KnowledgeBase(
    config_json=config_json, 
    user=username,
    connection_as_function=databricks_connector
    )

databricks_knowledge_agent = KnowledgeBaseAgent(
    agent_name = databricks_knowledge_agent_name,
    work_instance = agent_work,
    KnowledgeBase=doc_manager_databricks,
    k = 5,
    allow_direct_response = True,
    human_in_the_loop = False,
#   next_agent_list = ['colocar o nome dos agentes']  por padrão esse agente não delega tarefa para nenhum outro agente
)

✅ databricks_connector: Conectado ao Databricks
Manager iniciated


#### Analytics

In [ ]:
config_json = os.path.join("config_json", "databricks", "config_databricks_datasources.json")

In [ ]:
# Apenas necessário se ainda não existir config_file (se ja existir, esse comando será automaticamente ignorado)

DatabaseManager_Databricks.create_config_file(
    config_json=config_json,
    connection_server_hostname  = "adb-671799829240675.15.azuredatabricks.net",
    connection_http_path        = "/sql/1.0/warehouses/1fd972f888afd086",
    users_access_table_name     = "dtcore_prd.aida.acesso_usuario",
    users_access_profile_prefix = "GDB_DLKC_PRD_PFL_"
    )

In [ ]:
# Adiciona Coleções de Dados, quantas desejar (se já estiver cadastrada no config_json não precisa rodar)

DatabaseManager_Databricks.add_data_source(
    collection          = "intimacoes_sefaz_rj_pgov_gastos",
    process_description = "Rateio de gastos auditados pela SEFAZ-RJ relacionados às participações governamentais",
    relationships       = "A tabela 1 se relaciona com a 2 usando o campo abc",
    tables={
        "base_h"        : "dt0047_prd.gastos.base_h_pgov_sefaz_rj_gastos",
        "base_r"        : "dt0047_prd.gastos.base_r_pgov_sefaz_rj_gastos",
        "base_resultado": "dt0047_prd.gastos.base_result_pgov_sefaz_rj_gastos"
    }
)

DatabaseManager_Databricks.add_data_source(
    collection          = "Estoque_Produtos",
    process_description = "Análise das contas de estoques de produtos registradas na transação CKM3 para montagem do custeio e outras análises relacionadas.",
    relationships       = "",
    tables={
        "ckm3_custeio": "dt0047_prd.custeio.base_acumulada_custeio_ckm3"
    }
)

In [ ]:
dataanalytics_databricks = DatabaseManager_Databricks(
    config_json=config_json, 
    user=username,
    connection_as_function=databricks_connector
    )

databricks_analytics_agent = DatabaseExpertAgent(
    agent_name = databricks_analytics_agent_name,
    work_instance = agent_work,
    database_manager = dataanalytics_databricks,
    human_in_the_loop = False,
    allow_direct_response = True,
#   next_agent_list = ['colocar o nome dos agentes']  por padrão esse agente não delega tarefa para nenhum outro agente
)

❌ databricks_connector: Falha na conexão com Databricks → Error during request to server
❌ databricks_connector: Falha na conexão com Databricks → Error during request to server
❌ databricks_connector: Falha na conexão com Databricks → Error during request to server
❌ databricks_connector: Falha na conexão com Databricks → Error during request to server
❌ databricks_connector: Falha na conexão com Databricks → Error during request to server
❌ databricks_connector: Falha na conexão com Databricks → Error during request to server
❌ databricks_connector: Falha na conexão com Databricks → Error during request to server
❌ databricks_connector: Falha na conexão com Databricks → Error during request to server
❌ databricks_connector: Falha na conexão com Databricks → Error during request to server
❌ databricks_connector: Falha na conexão com Databricks → Error during request to server
❌ databricks_connector: Falha na conexão com Databricks → Error during request to server
❌ databricks_connecto

ConnectionError: ❌ Erro: Não foi possível conectar ao Databricks.

#### Teste Coleções

In [11]:
doc_manager_databricks.user = "CYGN"

In [12]:
doc_manager_databricks.get_available_collections(print_sql=True)

✅ databricks_connector: Conectado ao Databricks
KnowledgeBaseManager_Databricks: SQL:
 
        SELECT DISTINCT PEPA_CD_PERFIL_PLATAFORMA as profile_id, VWRP_CD_ELEMENTO as user
        FROM dtcore_prd.aida.acesso_usuario
        WHERE PEPA_CD_PERFIL_PLATAFORMA IN (999999,468,467,468,466,468,669)
        AND VWRP_CD_ELEMENTO = 'CYGN'
        
KnowledgeBaseManager_Databricks: result SQL:
[{'profile_id': 467, 'user': 'CYGN'}]
KnowledgeBaseManager_Databricks: result:
['intimacoes_sefaz_rj']


['intimacoes_sefaz_rj']

In [10]:
doc_manager_databricks.available_collections

['intimacoes_sefaz_rj', 'normas_contabeis', 'apontamento_hh_parcerias']

In [ ]:
dataanalytics_databricks.available_collections

['intimacoes_sefaz_rj_pgov_gastos', 'Estoque_Produtos']

### Agents Pessoais

In [16]:
update_personal_data = False # Alterar para usar as células de carga de dados

#### Personal_KnowledgeBase

In [17]:
config_json = local_path("config_json_personal", "config_personal_doc_collection.json")
db_path     = local_path('data_base', "zdb_personal_collection.db")

In [18]:
# Apenas necessário se ainda não existir config_file (se ja existir, esse comando será automaticamente ignorado)
PersonalCollection.create_config_file(config_json =config_json, db_path=db_path)

# Cria instância para a classe PersonalCollection
Personal_Collection = PersonalCollection(config_json = config_json)

Manager iniciated


In [19]:
if update_personal_data:
        
    # Adiciona Coleções de Dados, quantas desejar
    nome_da_colecao = "Pessoal"

    Personal_Collection.create_collection(
        collection_name   = nome_da_colecao,
        table_common_name = "Dados de agendamento",
        curator           = "eu",
        description       = "meus documentos sobre requisições e agendamentos",
        gpt_instructions  = "sem relacionamentos"
        )

    file = r"C:\Users\zcfv\OneDrive - PETROBRAS\Desktop\MEST_CIEN_COMPUT_031_2023_retif.pdf"

    Personal_Collection.upload_document(
        collection_name     = nome_da_colecao,
        conflict_option     = "ignore",
        file_path           = file,
        read_file_class     = read_file, # Usa a classe read files da pasta READERS
        llm_client          = BaseAgent.llm,
        words_per_chunk     = 200,
        overlap             = 15,
        metadata_dictionary = {}
        )

In [20]:
personal_knowledge_agent = KnowledgeBaseAgent(
    agent_name = personal_knowledge_agent_name,
    work_instance = agent_work,
    KnowledgeBase = None,
    k = 5,
    human_in_the_loop = False,
    allow_direct_response = True,
#   next_agent_list = ['colocar o nome dos agentes']  por padrão esse agente não delega tarefa para nenhum outro agente    
)

#### Personal_Analytics

In [21]:
config_json = local_path("config_json_personal", "config_personal_datasources.json")
db_path     = local_path('data_base', "zdb_personal_datasources.db")

In [22]:
# Apenas necessário se ainda não existir config_file (se ja existir, esse comando será automaticamente ignorado)
DatabaseManager_SQLite.create_config_file(config_json =config_json, db_path=db_path)

# Cria instância para a classe PersonalCollection
dataanalytics_local = DatabaseManager_SQLite(config_json = config_json)

exists


In [23]:
if update_personal_data:
    # Adiciona Coleções de Dados, quantas desejar
    nome_da_colecao = "sisrend"
        
    dataanalytics_local.add_datasource(
        data_source         = nome_da_colecao,
        process_description = "Análise dos lançamentos de arrendamento mercantil (IFRS 16)",
        relationships       = ""
    )

    # Fazer o upload do arquivo (subistitui se table_name já existir no data_source)
    dataanalytics_local.load_csv_to_table(
        data_source         = nome_da_colecao,
        table_name          = "razão",
        conflict_option     = "replace",
        file_path           = r"upload_files/sisrend/202201.csv",
        column_type         = {"Data de lançamento": "DATE"}
    )

    # Fazer o upload do arquivo (subistitui se table_name já existir no data_source)
    dataanalytics_local.load_csv_to_table(
        data_source         = nome_da_colecao,
        table_name          = "razão",
        conflict_option     = "append",
        file_path           = r"upload_files\sisrend\202202.csv",
    )
    
    # # Adiciona Coleções de Dados, quantas desejar
    # nome_da_colecao = "CKM3"
        
    # dataanalytics_local.add_datasource(
    #     data_source         = nome_da_colecao,
    #     process_description = "Análise do estoque de produtos utilizando os dados da movimentação de estoque obtidos pela transação CKM3",
    #     relationships       = ""
    # )

    # # Fazer o upload do arquivo (subistitui se table_name já existir no data_source)
    # table_name = ""
    # if table_name == "":
    #     table_name=None
    # file = "upload_files\CKM3_tdv_042024.csv"

    # csv_file = open(file, "rb")
    # csv_file = {
    #         'file_content': csv_file.read(),
    #         'file_name': "CKM3_tdv_042024.csv"
    #     }
    # dataanalytics_local.load_csv_to_table(
    #     data_source         = nome_da_colecao,
    #     # conflict_option     = "ignore",
    #     csv_file            = csv_file,
    #     # file_path           = file,
    #     table_name          = table_name,
    #     # column_type         = { # Apenas se for necessário forçar o tipo de dados das colunas
    #     #     "Metade do valor (Moeda 10)": "float64"
    #     # }
    # )

In [24]:
personal_analytics_agent = DatabaseExpertAgent(
    agent_name            = personal_analytics_agent_name,
    work_instance = agent_work,
    database_manager      = dataanalytics_local,
    human_in_the_loop     = False,
    allow_direct_response = True,
#   next_agent_list = ['colocar o nome dos agentes']  por padrão esse agente não delega tarefa para nenhum outro agente    
)

## Chat

In [29]:
agent_work.chat_mode = False

In [27]:
agent_work.crew_work("bom dia, me chamo marcos")

BaseAgent: crew_work
08e02f13-af91-433a-8c42-9e8c8d96e958
BaseAgent: Agent: CHAT ANALISTA

llmClient_AzureOpenAI: message to llm: [{'role': 'system', 'content': '{\n  "agent_description": {\n    "agent_name": "Chat Analista",\n    "collection": [],\n    "background": [\n      "Seu conhecimento está limitado a um período no tempo, quando você parou de ser treinado.",\n      "Por esse motivo, perguntas que envolvam dados atuais nunca devem ser respondidas com base no seu conhecimento.",\n      "Além disso, seu conhecimento não comtempla as informações internas, conhecida por outros agentes."\n    ],\n    "goal": [\n      "Responda diretamente perguntas simples de saudações.",\n      "De preferencia para respostas que se baseiem no conhecimento interno, que são dominio de outros agentes.",\n      "Não sendo o caso, avalie se é necessário o uso de alguma ferramenta ou se deve passar logo a tarefa para outro agente.",\n      "Quando usar uma ferramenta, não mencione que ela foi usada, apena

In [ ]:
agent_work.crew_work("que horas são?")

crew_work


PythonTools can run arbitrary code, please provide human supervision.


In [32]:
BaseAgent.crew_work("qual o meu nome")

crew_work
Agent: CHAT ANALISTA

    Executando tool: direct_anwser

    tool: {'query_input': 'Bom dia'}

answer: Bom dia



In [ ]:
BaseAgent.crew_work("qual o meu nome")

Agent: CHAT ANALISTA

    Executando tool: direct_anwser

    tool: {'query_input': 'qual o meu nome'}

answer: Seu nome é Marcos.



In [ ]:
BaseAgent.chat_memory.chat_id

'b45ed62c-b6e9-4d2a-86eb-5953075ac221'

In [12]:
agent_work.crew_work("""
o que foi respondido a intimaçãoes na RAF 55668100
""")

BaseAgent: crew_work
fb600e59-fe06-4c86-a3a3-d0cf69b7804e
BaseAgent: Agent: CHAT ANALISTA

llmClient_AzureOpenAI: message to llm: [{'role': 'system', 'content': '{\n  "agent_description": {\n    "agent_name": "Chat Analista",\n    "collection": [],\n    "background": [\n      "Seu conhecimento está limitado a um período no tempo, quando você parou de ser treinado.",\n      "Por esse motivo, perguntas que envolvam dados atuais nunca devem ser respondidas com base no seu conhecimento.",\n      "Além disso, seu conhecimento não comtempla as informações internas, conhecida por outros agentes."\n    ],\n    "goal": [\n      "Responda diretamente perguntas simples de saudações.",\n      "De preferencia para respostas que se baseiem no conhecimento interno, que são dominio de outros agentes.",\n      "Não sendo o caso, avalie se é necessário o uso de alguma ferramenta ou se deve passar logo a tarefa para outro agente.",\n      "Quando usar uma ferramenta, não mencione que ela foi usada, apena

In [ ]:
BaseAgent.crew_work("""
que horas são
                    """)

Agent: CHAT ANALISTA



PythonTools can run arbitrary code, please provide human supervision.


    Executando tool: by_code

    tool: {'code': "from datetime import datetime\ncurrent_time = datetime.now().strftime('%H:%M:%S')", 'variable_to_return': 'current_time'}

answer: São 18:54:30.



In [ ]:
BaseAgent.crew_work("""
para morar
                    """)

CHAT ANALISTA

    Executando tool: remake_user_question
    {'query_input': 'Escolha uma cidade para morar', 'motivation': 'A pergunta original estava incompleta e vaga.'}


answer: Para qual tipo de estilo de vida ou características você está procurando ao escolher uma cidade para morar?



In [ ]:
BaseAgent.crew_work("""
com boa empregabilidade
                    """)

CHAT ANALISTA

    Executando tool: remake_user_question
    {'query_input': 'com boa empregabilidade', 'motivation': 'A pergunta está vaga e precisa de mais contexto para fornecer uma resposta adequada.'}


answer: Para qual tipo de estilo de vida ou características você está procurando ao escolher uma cidade para morar com boa empregabilidade?



In [ ]:
BaseAgent.crew_work("""
que horas sao
                    """)

CHAT ANALISTA



PythonTools can run arbitrary code, please provide human supervision.


    Executando tool: by_code
    {'code': "from datetime import datetime\ncurrent_time = datetime.now().strftime('%H:%M:%S')", 'variable_to_return': 'current_time'}


answer: São 15:16:48.



In [ ]:
BaseAgent.crew_work("""
me diga 3 festas no rio de janeiro para ir na próxima sexta feira, hora e e local de cada uma
                                        """)

In [ ]:
BaseAgent.crew_work("""
quero uma lista com festas reais
                                        """)

## Testes

In [21]:
doc_manager_databricks

In [22]:
copy.deepcopy(doc_manager_databricks)

In [20]:
teste['a']="f"

In [17]:
print(databricks_connector_global)
teste = copy.deepcopy(databricks_connector_global)
print(teste)

<function databricks_connector_global at 0x000001B6330E4540>
<function databricks_connector_global at 0x000001B6330E4540>


In [25]:
from Petrobras_AI_Agents import AppSessionManager_Postgres as AppSessionManager

global_session_db = AppSessionManager(db_path="s10883_aplicacao:nMAEzdHlVX@bdpgsqld.petrobras.com.br:5432/s10883d")

In [30]:
agent_work.chat_memory.chat_id

'cd0d109b-7b8b-4d1a-8e2a-fed090a64f6f'

In [26]:
global_session_db.save_sessiont('testes', 'agent_work', agent_work)

In [33]:
agent_work.chat_memory.create_new_chat('testes', 'en')

In [34]:
agent_work.chat_memory.chat_id

'c9ab2116-0d73-4d8e-b080-e6a599232b29'

In [36]:
agent_work2 = global_session_db.load_session('testes', 'agent_work')

In [37]:
agent_work2.chat_memory.chat_id

'cd0d109b-7b8b-4d1a-8e2a-fed090a64f6f'

In [20]:
a = 3
print(b)
print(a)

2
3


In [1]:
from Petrobras_AI_Agents import databricks_connector

In [2]:
databricks_conn = databricks_connector(
    server_hostname="adb-671799829240675.15.azuredatabricks.net",
    http_path="/sql/1.0/warehouses/1fd972f888afd086",
    access_token=
)

❌ databricks_connector: Falha na conexão com Databricks → Error during request to server


In [26]:
import dill
import base64
base64.b64encode(dill.dumps(databricks_connector)).decode('utf-8')

'gASVRgAAAAAAAACMKVBldHJvYnJhc19BSV9BZ2VudHMuQ09OTkVDVE9SUy5kYXRhYnJpY2tzlIwUZGF0YWJyaWNrc19jb25uZWN0b3KUk5Qu'

In [21]:
import weakref
conn = weakref.ref(databricks_conn)

In [22]:
conn

<weakref at 0x0000010992981CB0; to 'Connection' at 0x0000010990B3DC50>

In [23]:
with conn.cursor() as cursor:
                    cursor.execute("SELECT 1")
                    result = cursor.fetchall()
                    print(result)

AttributeError: 'weakref.ReferenceType' object has no attribute 'cursor'

In [26]:
agent_work.__dict__

{'language': 'pt-br',
 'llm': <Petrobras_AI_Agents.LLM.azure_openai.llmClient_AzureOpenAI at 0x1f0376f3c10>,
 'chat_memory': <Petrobras_AI_Agents.MEMORY.postgres.ChatDatabaseManager_Postgres at 0x1f0693bd250>,
 'start_agent': 'Chat Analista',
 'chat_mode': False,
 'short_memory_size': 10,
 'agents': {'Chat Analista': {'agent': <Petrobras_AI_Agents.AGENTS.chatinterpretor.ChatInterpretorAgent at 0x1f035e7a610>,
   'active': True},
  'Documentos Databricks': {'agent': <Petrobras_AI_Agents.AGENTS.knowledge_agent.KnowledgeBaseAgent at 0x1f0376ccdd0>,
   'active': True},
  'Analytics Databricks': {'agent': <Petrobras_AI_Agents.AGENTS.dataanalysis_agent.DatabaseExpertAgent at 0x1f069720790>,
   'active': True},
  'Documentos Pessoais': {'agent': <Petrobras_AI_Agents.AGENTS.knowledge_agent.KnowledgeBaseAgent at 0x1f069786050>,
   'active': False},
  'Analytics Pessoal': {'agent': <Petrobras_AI_Agents.AGENTS.dataanalysis_agent.DatabaseExpertAgent at 0x1f069796410>,
   'active': False}},
 'agemt

In [40]:
agent_work.agents['Analytics Databricks']['agent'].database_manager.__dict__

{'connection': <databricks.sql.client.Connection at 0x1f0691fa150>,
 '_profile_table_list': [{'profile_id': 999999, 'address': ''},
  {'profile_id': 191,
   'address': 'dt0047_prd.gastos.base_h_pgov_sefaz_rj_gastos'},
  {'profile_id': 361,
   'address': 'dt0047_prd.gastos.base_h_pgov_sefaz_rj_gastos'},
  {'profile_id': 191,
   'address': 'dt0047_prd.gastos.base_r_pgov_sefaz_rj_gastos'},
  {'profile_id': 361,
   'address': 'dt0047_prd.gastos.base_r_pgov_sefaz_rj_gastos'},
  {'profile_id': 191,
   'address': 'dt0047_prd.gastos.base_result_pgov_sefaz_rj_gastos'},
  {'profile_id': 361,
   'address': 'dt0047_prd.gastos.base_result_pgov_sefaz_rj_gastos'},
  {'profile_id': 411,
   'address': 'dt0047_prd.custeio.base_acumulada_custeio_ckm3'},
  {'profile_id': 191,
   'address': 'dt0047_prd.custeio.base_acumulada_custeio_ckm3'}],
 'config_json': 'config_json\\databricks\\config_databricks_datasources.json',
 '_allowed_tables': None,
 'config': {'connection': {'server_hostname': 'adb-67179982924

In [13]:
import dill
import base64
user_BaseAgent = base64.b64encode(dill.dumps(agent_work)).decode('utf-8')

In [30]:
BaseAgent.crew_work("""
o que foi respondido a intimaçãoes na RAF 55668100
""")

crew_work
eef99fa1-0441-43c2-8475-61fc8606dfdc
Agent: CHAT ANALISTA

    Executando tool: remake_user_question

    tool param: {'query_input': 'o que foi respondido a intimaçãoes na RAF 55668100', 'motivation': 'A pergunta precisa de mais contexto ou detalhes específicos para ser respondida corretamente.'}

answer: Para responder à sua pergunta sobre o que foi respondido às intimações relacionadas ao caso RAF 55668100, vou precisar consultar a base de dados de intimações da SEFAZ-RJ.

Agent: DOCUMENTOS DATABRICKS

    Executando tool: rag_complement

    tool param: {'query_input': 'o que foi respondido a intimaçãoes na RAF 55668100'}

    Executando tool: _get_collection_name

    tool result: intimacoes_sefaz_rj

    Executando tool: _generate_sql_with_filters

    tool result: {'query': 'SELECT COUNT(*) AS count FROM dt0047_prd.chat_bot.intimacoes_fiscais_pgov_sefaz_rj_file_base AS file_base', 'filters': ["file_base.raf = '55668100'", "file_base.raf LIKE '55668100%'"]}

    Executa